### Nominal Stress approach ###
![Bosch House of Reliability](pic/HouseOfReliability.png)
This Notebook shows a general calculation stream for a nominal stress reliability approach. 
#### Stress derivation #####
First, on the stress side we read in different time signals (coming from a test bench or a vehicle measurement e.g.). 

1. Import the time series into a pandas Data Frame
2. Resample the time series if necessary
3. Filter the time series with a bandpass filter if necessary
4. Edititing time series using Running Statistics methods 
5. Rainflow Calculation
6. Mean stress correction
7. Multiplication with repeating factor of every time series 

#### Strength derivation #####
1. Import the SN data
2. Define load cycle limit
3. Select the material cycle parameter calculation method
4. Plot and check



In [ ]:
'''Here we read in all necessary libraries'''
import numpy as np
import pandas as pd

from pylife.stress.rainflow import *
import pylife.strength.meanstress
import pylife.stress.timesignal as ts
from pylife.strength import miner
from pylife.strength.miner import MinerElementar, MinerHaibach
import matplotlib.pyplot as plt
from pylife.materialdata.woehler.diagram import *
from pylife.materialdata.woehler.widgets import *
from pylife.materialdata.woehler.analyzer import *
import pylife.utils.meshplot
import pylife.stress.equistress
from scipy.stats import norm
from pylife.strength import failure_probability as fp
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import io
from IPython.display import HTML
import base64 

### Stress derivation ###

#### 1. Import the time series into a pandas Data Frame #####

In [ ]:
files = ["IIS0042_DAT_2017LK065HU004_000100_0206_ESP_FA_arf_0004163.dat.csv",
         "IIS0042_DAT_2017LK065HU004_000100_0206_ALC7_abf_0004587.dat.csv",
         "IIS0042_DAT_2017LK065HU004_000100_0205_hfc2_abf_0003880.dat.csv"]
data_loc = '\\\\fe00fs76.de.bosch.com\\SP17_Extern$\\SP17-010\\TSIPRAS\\MeasuredData\\Messdaten_ESP_Maneuver\\'

In [ ]:
input_data = []
for upload in files:
    data_akt = pd.read_csv(data_loc + upload, sep = ",")
    if len(data_akt.columns) == 1:
        print ('please use "," as seperator next time')
        data_akt = pd.read_csv(data_loc + upload, sep = ";")
    input_data.append(data_akt)
    print(upload + " imported succesfully")

#### 2. Resample the time series if necessary

There is also a plot option

In [ ]:
f_resample = widgets.FloatText(value = 40e3,min=1,max=100e3,step=10,
    description='Resampling frequency [Hz]',
    disabled=False,readout=True,readout_format='d')
display(f_resample)
# select time column
timeColumn = widgets.Dropdown(options = data_akt.columns)
display(timeColumn)

In [ ]:
print(f_resample.value)
print(timeColumn.value)
meas_resample = []
for file_act in input_data:
    meas_resample.append(ts.TimeSignalPrep(file_act).resample_acc(timeColumn.value,f_resample.value))

In [ ]:
print("select channel to plot")
plotChan = widgets.Dropdown(options = file_act.drop([timeColumn.value],axis = 'columns').columns)
display(plotChan)

In [ ]:
%matplotlib notebook
fig, ax = plt.subplots(len(meas_resample))
fig.suptitle('Resampled input data')
ii = 0
for df_act in meas_resample:
    ax[ii].plot(df_act['t'], df_act[plotChan.value])
    ii += 1

#### 3. Filter the time series with a bandpass filter if necessary

In [ ]:
f_min = widgets.FloatText(value = 100,description='min frequency [Hz]',disabled=False)
f_max = widgets.FloatText(value = 5e3,description='max frequency [Hz]',disabled=False)
display(f_min)
display(f_max)

In [ ]:
bandpass = []
for df_act in meas_resample:
    bandpassDF = df_act[[timeColumn.value]]
    for col_act in df_act.drop([timeColumn.value],axis = 'columns').columns:
        bandpassDF[col_act] = ts.TimeSignalPrep(df_act.loc[:,col_act]).butter_bandpass(f_min.value,f_max.value,f_resample.value,5)
    bandpass.append(bandpassDF)            

#### 4. Edititing time series using Running Statistics methods 

In [ ]:
print("select channel to for running stats")
runChan = widgets.Dropdown(options = df_act.drop([timeColumn.value],axis = 'columns').columns)
display(runChan)
print(" Running statistics method")
method_choice = widgets.Dropdown(options = ['rms','max','min','abs'])
display(method_choice)

paraRunStats = ['window_length', 'buffer_overlap', 'limit']
values = [800,0.1,0.1]
child = [widgets.FloatText(description=name) for name in paraRunStats]
tab = widgets.Tab()
tab.children = child
for i in range(len(child)):
    tab.set_title(i, paraRunStats[i])
    tab.children[i].value = values[i]

tab

In [ ]:
""" Running statistics to drop out zero values """
cleaned = []
for df_act in bandpass:
    cleaned_df = ts.TimeSignalPrep(df_act).running_stats_filt(col = runChan.value,
                            window_length = int(tab.children[0].value),buffer_overlap = int(tab.children[1].value),limit = tab.children[2].value,method = method_choice.value)
    cleaned_df[timeColumn.value] =  np.linspace(0,len(cleaned_df)/f_resample.value,len(cleaned_df)+1)[:-1]          
    cleaned.append(cleaned_df)

In [ ]:
%matplotlib notebook
fig, ax = plt.subplots(len(meas_resample))
fig.suptitle('Cleaned input data')
ii = 0
for df_act in cleaned:
    ax[ii].plot(df_act['t'], df_act['PCB_P93_A1S_5116_Z'])
    ii += 1    

#### 5. Rainflow Calculation

In [ ]:
rfcChan = widgets.Dropdown(options = df_act.drop([timeColumn.value],axis = 'columns').columns)
display(rfcChan)

In [ ]:
%matplotlib notebook
rainflow = []
for df_act in cleaned:
    rfc = RainflowCounterFKM().process(df_act[rfcChan.value].values)
    rfm = rfc.get_rainflow_matrix_frame(64)
    rainflow.append(rfm)

In [ ]:
%matplotlib notebook
fig, ax = plt.subplots(1,len(rainflow))
fig.suptitle('Rainflow of Channel ' + rfcChan.value)
ii = 0
for rf_act in rainflow:
    froms = rf_act.index.get_level_values('from').mid
    tos = rf_act.index.get_level_values('to').mid
    ax[ii].set_xlabel('From')
    ax[ii].set_ylabel('To')
    ax[ii].imshow(np.flipud(np.log10(rf_act.values.reshape(rf_act.index.levshape).T)), extent=[froms.min(), froms.max(), tos.min(), tos.max()])
    ii += 1    


#### 6. Mean stress correction

In [ ]:
meanstress_para = ['M', 'M2', 'R_Goal']
values = [0.3,0.2,-1]
child = [widgets.FloatText(description=name) for name in meanstress_para]
tab_mean = widgets.Tab()
tab_mean.children = child
for i in range(len(child)):
    tab_mean.set_title(i, meanstress_para[i])
    tab_mean.children[i].value = values[i]

tab_mean

In [ ]:
transformed = []
for rf_act in rainflow:
    transformed.append(rf_act.meanstress_hist.FKM_goodman(pd.Series({'M': tab_mean.children[0].value, 
                                                                     'M2': tab_mean.children[1].value})
                                                          , R_goal = tab_mean.children[2].value))

#### 7. Multiplication with repeating factor of every time series 

In [ ]:
child = [widgets.FloatText(description=name) for name in files]
tab_repeat = widgets.Tab()
tab_repeat.children = child
for i in range(len(child)):
    tab_repeat.set_title(i, files[i])
    tab_repeat.children[i].value = 1

tab_repeat

In [ ]:
for ii in range(3): 
    transformed[ii] = transformed[ii]*tab_repeat.children[ii].value

## Nominal stress approach ##

### Material parameters ###

In [ ]:
mat = WL_param()
display(mat)

In [ ]:
mat_para, _ = WL_param_display(mat)
sn_curve_parameters =  {key:mat_para[key] for key in mat_para.keys() - {'TN_inv', 'TS_inv'}} 
mat_para['TS_inv'] = mat_para['TN_inv']**(1/mat_para['k_1'])
print(mat_para)
print(sn_curve_parameters)

### Damage Calculation ###

In [ ]:
SNmethod = widgets.Dropdown(options = ['Miner Elementar','Miner Haibach','Miner original'])
display(SNmethod)

In [ ]:
damage_calc = MinerElementar(**sn_curve_parameters)
data_act = transformed[0]
range_mid = data_act.index.get_level_values('range').mid.values
data_finite = data_act[range_mid > sn_curve_parameters['SD_50']][::-1]
stress_finite = data_finite.index.get_level_values('range').mid.values
N = damage_calc.sn_curve.calc_N(stress_finite)
d = np.sum(data_act[range_mid > sn_curve_parameters['SD_50']].values/N)
print("\033[1m  Total Damage: %.2f  \033[0m" %d) 

In [ ]:
%matplotlib notebook
class wc_data:
    def __init__(self, k, TN,loads_max):
        self.k = k
        self.TN = TN
        self.loads_max = loads_max
SRI = mat_para['SD_50']*(mat_para['ND_50']**(1/mat_para['k_1']))
wc = wc_data(mat_para['k_1'],mat_para['TN_inv'],SRI) 
_ = PlotWoehlerCurve.final_curve_plot(wc, mat_para['SD_50'], mat_para['ND_50'],mat_para['TS_inv'],0, 
                                      None,"Amplitude","Load", " ", (1,1e8), (1e1,1e3), 0)
plt.barh(stress_finite,np.cumsum(data_finite.values), height = data_finite.index.get_level_values('range').length.min())

## Failure Probaility ##

#### Without field scatter ####

In [ ]:
%matplotlib notebook
D50 = 0.3
mat_std = np.log10(mat_para['TN_inv'])/2.5631031311
dmin = d/10
di = np.linspace(dmin,50*d,1000)
failprob = fp.FailureProbability(D50,mat_std).pf_simple_load(di)
#print(failprob)
fig, ax = plt.subplots()
ax.semilogx(di, failprob, label='cdf')
ax.vlines(d, 0, fp.FailureProbability(D50,mat_std).pf_simple_load(d))
# 
ix = np.linspace(dmin, d)
ax.fill_between(ix, fp.FailureProbability(D50,mat_std).pf_simple_load(ix), y2=0,color = 'y')
plt.xlabel("Damage")
plt.ylabel("cdf")
plt.title("Failure probability = %.2f" %fp.FailureProbability(D50,mat_std).pf_simple_load(d))  
plt.ylim(0,1)

#### With field scatter ####

In [ ]:
field_std = 0.35
fig, ax = plt.subplots()
# plot pdf of material
mat_pdf = norm.pdf(np.log10(di), loc=np.log10(D50), scale=mat_std)
ax.semilogx(di, mat_pdf, label='pdf_mat')
# plot pdf of load
field_pdf = norm.pdf(np.log10(di), loc=np.log10(d), scale=field_std)
ax.semilogx(di, field_pdf, label='pdf_load',color = 'r')
# area_1
area = np.minimum(mat_pdf, field_pdf)
ax.fill_between(di, area, y2=0,color = 'y')
plt.xlabel("Damage")
plt.ylabel("pdf")
plt.title("Failure probability = %.2f" %fp.FailureProbability(D50,mat_std).pf_norm_load(d,field_std))  
plt.legend()
